In [1]:
!pip install tabulate

In [1]:
import numpy as np
from tabulate import tabulate


In [ ]:
# Scenario 1: No Aggregation

In [2]:
# Given data
demands = np.array([1000, 300, 100, 50])
ordering_costs = np.array([110, 120, 125, 125]) 
unit_costs = np.array([50, 60, 30, 30])
holding_cost_rates = np.array([0.2, 0.2, 0.2, 0.2])
annual_demand = np.sum(demands)

# Initialize lists to store results
Optimal_Q = []
T = []
N = []
Holding_Cost = []

# Calculation loop
for i in range(4):
    D = demands[i]
    S = ordering_costs[i]
    H = holding_cost_rates[i] * unit_costs[i]
    Q = np.sqrt((2 * D * S) / H)
    Optimal_Q.append(Q)
    T_i = Q / D
    T.append(T_i)
    N_i = 1 / T_i
    N.append(N_i)
    Holding_Cost_i = (Q / 2) * H
    Holding_Cost.append(Holding_Cost_i)

# Store result in table format
opt_results = [["Product {}".format(i+1), Optimal_Q[i], T[i], N[i], Holding_Cost[i]] for i in range(4)]
headers = ["Product", "Optimal Q", "Time Between Orders (T)", "Order Frequency (N)", "Holding Cost"]

# Print the table
print(tabulate(opt_results, headers=headers, tablefmt="pretty", numalign="right"))

# Annual Cost Calculation
Ordering_Cost = sum(demands / Optimal_Q * ordering_costs)
Total_Holding_Cost = sum(Holding_Cost)  # Already calculated in the loop

Total_Annual_Cost_1 = Ordering_Cost + Total_Holding_Cost

print(f"Total Annual Cost: ${Total_Annual_Cost_1:.2f}")

+-----------+--------------------+-------------------------+---------------------+--------------------+
|  Product  |     Optimal Q      | Time Between Orders (T) | Order Frequency (N) |    Holding Cost    |
+-----------+--------------------+-------------------------+---------------------+--------------------+
| Product 1 | 148.32396974191326 |   0.14832396974191325   |  6.741998624632421  | 741.6198487095663  |
| Product 2 | 77.45966692414834  |   0.2581988897471611    | 3.8729833462074175  |  464.75800154489   |
| Product 3 | 64.54972243679029  |   0.6454972243679029    | 1.5491933384829666  | 193.64916731037087 |
| Product 4 | 45.64354645876384  |   0.9128709291752769    | 1.0954451150103321  | 136.93063937629154 |
+-----------+--------------------+-------------------------+---------------------+--------------------+
Total Annual Cost: $3073.92


In [ ]:
# Scenario 2: Complete Aggregation

In [3]:
# Common ordering cost S, using the average of the provided ordering costs for simplification
S = np.mean(ordering_costs)

# Calculate the numerator of the formula for n
numerator = sum(demands * holding_cost_rates * unit_costs)

# Calculate n for complete aggregation
n = np.sqrt(numerator / (2 * S))

# Calculate total holding cost and ordering cost under complete aggregation
# Assuming optimal order quantity Q for each product based on n
Optimal_Q = demands / n
Total_Holding_Cost = sum((Optimal_Q / 2) * holding_cost_rates * unit_costs)
Total_Ordering_Cost = n * S


# Total Annual Cost
Total_Annual_Cost_2 = Total_Ordering_Cost + Total_Holding_Cost

# Display results
print(f"Order Frequency (n) under Complete Aggregation: {n:.2f}")
print(f"Total Annual Cost under Complete Aggregation: ${Total_Annual_Cost_2:.2f}")

# For displaying each product's details in a table
opt_results_aggregated = [["Product {}".format(i+1), Optimal_Q[i], demands[i] / Optimal_Q[i], n, (Optimal_Q[i] / 2) * holding_cost_rates[i] * unit_costs[i]] for i in range(4)]
headers_aggregated = ["Product", "Optimal Q", "Time Between Orders (T)", "Order Frequency (N)", "Holding Cost"]

# Print the table for Complete Aggregation
print(tabulate(opt_results_aggregated, headers=headers_aggregated, tablefmt="pretty", numalign="right"))

Order Frequency (n) under Complete Aggregation: 7.77
Total Annual Cost under Complete Aggregation: $1865.48
+-----------+--------------------+-------------------------+---------------------+--------------------+
|  Product  |     Optimal Q      | Time Between Orders (T) | Order Frequency (N) |    Holding Cost    |
+-----------+--------------------+-------------------------+---------------------+--------------------+
| Product 1 | 128.65350418053538 |    7.772815877574013    |  7.772815877574012  | 643.2675209026769  |
| Product 2 | 38.596051254160614 |    7.772815877574013    |  7.772815877574012  | 231.5763075249637  |
| Product 3 | 12.865350418053538 |    7.772815877574013    |  7.772815877574012  | 38.596051254160614 |
| Product 4 | 6.432675209026769  |    7.772815877574013    |  7.772815877574012  | 19.298025627080307 |
+-----------+--------------------+-------------------------+---------------------+--------------------+


In [ ]:
# Scenario 3: Tailored Aggregation

In [4]:
# Given data
demands = np.array([1000, 300, 100, 50])
specific_ordering_costs = np.array([10, 20, 25, 25])
common_ordering_cost = 100
unit_costs = np.array([50, 60, 30, 30])
holding_cost_rates = np.array([0.2, 0.2, 0.2, 0.2])

# Step 1: Calculate independent ordering frequencies for each product (assuming EOQ)
independent_frequencies = np.sqrt((2 * demands * specific_ordering_costs) / (holding_cost_rates * unit_costs))

# Identify the most frequently ordered product (highest frequency or demand)
most_frequently_ordered_index = np.argmax(independent_frequencies)
most_frequently_ordered_demand = demands[most_frequently_ordered_index]

# Step 2 & 3: Recompute frequencies for all products using their specific ordering costs and determine mi
mi = np.ceil(demands / most_frequently_ordered_demand)

# Step 4: Recalculate ordering frequency n for the most frequently ordered product
numerator = np.sum(holding_cost_rates * unit_costs * demands * mi)
denominator = 2 * (common_ordering_cost + np.sum(specific_ordering_costs * mi))
n = np.sqrt(numerator / denominator)

# Calculate the adjusted order quantities based on the recalculated frequency n
adjusted_order_quantities = demands / n

# Calculate total costs
Ordering_Cost = n * common_ordering_cost  # Assuming one order per cycle for all products together
Holding_Cost = np.sum((adjusted_order_quantities / 2) * holding_cost_rates * unit_costs)

Total_Annual_Cost_3 = Ordering_Cost + Holding_Cost

# Output results
print(f"Recalculated Order Frequency (n): {n}")
print(f"Adjusted Order Quantities: {adjusted_order_quantities}")
print(f"Total Annual Cost: ${Total_Annual_Cost_3:.2f}")

Recalculated Order Frequency (n): 6.346477588219924
Adjusted Order Quantities: [157.56771943  47.27031583  15.75677194   7.87838597]
Total Annual Cost: $1777.01


In [5]:
# Create a list of lists for the table data
comparison_data = [
    ["No Aggregation", f"${Total_Annual_Cost_1:,.2f}"],
    ["Complete Aggregation", f"${Total_Annual_Cost_2:,.2f}"],
    ["Tailored Aggregation", f"${Total_Annual_Cost_3:,.2f}"],
]

# Define headers for the table
headers = ["Scenario", "Total Annual Cost"]

# Create and print the table
print(tabulate(comparison_data, headers=headers, tablefmt="pretty"))

+----------------------+-------------------+
|       Scenario       | Total Annual Cost |
+----------------------+-------------------+
|    No Aggregation    |     $3,073.92     |
| Complete Aggregation |     $1,865.48     |
| Tailored Aggregation |     $1,777.01     |
+----------------------+-------------------+
